In [13]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ExponentialLR

import pytorch_lightning as pl

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib
from matplotlib.ticker import MaxNLocator

from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.functional import accuracy

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import datetime
import time
import random
import os

# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda:0

NVIDIA GeForce RTX 2080 SUPER
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [14]:
Concatnated = pd.read_csv('Concatnated.csv')    #.set_index('Time')
Concatnated = np.array(Concatnated[:292980], dtype=np.float32)
#Concatnated = torch.from_numpy(Concatnated)
print(Concatnated.shape)

(292980, 447)


In [15]:
CreatData = []
X = Concatnated[:, :446]
y = Concatnated[:,446]
strat_time = 0
timestamp = 10
length = len(Concatnated)

for i in range(int(abs(length/10-1))):
    features = X[strat_time:timestamp, :446]
    features = np.reshape(features, (1, 10, 446))
    CreatData.append((features, y[timestamp]))

    strat_time = strat_time + 10
    timestamp = timestamp + 10

In [16]:
train_dataset, test_dataset = train_test_split(CreatData, test_size=0.2)
print("Number of Training Sequences: ", len(train_dataset))
print("Number of Testing Sequences: ", len(test_dataset))

Number of Training Sequences:  23437
Number of Testing Sequences:  5860


In [17]:
def set_seed(seed=0):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(0)

In [18]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 128, kernel_size = 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(in_channels = 128, out_channels = 64, kernel_size = 3)
        
        self.fc1 = nn.Linear(in_features = 64*1*110, out_features = 120)
        self.fc2 = nn.Linear(in_features = 120, out_features = 84)
        self.out = nn.Linear(in_features = 84, out_features = 4)                   

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = x.view(x.size(0), -1)           
        x = F.relu(self.fc1(x))               
        x = F.relu(self.fc2(x))               
        x = self.out(x)                      
        return x

In [19]:
for name, param in ConvNet().named_parameters():
    print(name, '\t\t', param.shape)

conv1.weight 		 torch.Size([128, 1, 3, 3])
conv1.bias 		 torch.Size([128])
conv2.weight 		 torch.Size([64, 128, 3, 3])
conv2.bias 		 torch.Size([64])
fc1.weight 		 torch.Size([120, 7040])
fc1.bias 		 torch.Size([120])
fc2.weight 		 torch.Size([84, 120])
fc2.bias 		 torch.Size([84])
out.weight 		 torch.Size([4, 84])
out.bias 		 torch.Size([4])


In [20]:
class EarlyStopping:
    def __init__(self, mode, path, patience=3, delta=0):
        if mode not in {'min', 'max'}:
            raise ValueError("Argument mode must be one of 'min' or 'max'.")
        if patience <= 0:
            raise ValueError("Argument patience must be a postive integer.")
        if delta < 0:
            raise ValueError("Argument delta must not be a negative number.")
            
        self.mode = mode
        self.patience = patience
        self.delta = delta
        self.path = path
        self.best_score = np.inf if mode == 'min' else -np.inf
        self.counter = 0
        
    def _is_improvement(self, val_score):
        """Return True iff val_score is better than self.best_score."""
        if self.mode == 'max' and val_score > self.best_score + self.delta:
            return True
        elif self.mode == 'min' and val_score < self.best_score - self.delta:
            return True
        return False
        
    def __call__(self, val_score, model):
        """Return True iff self.counter >= self.patience.
        """
        
        if self._is_improvement(val_score):
            self.best_score = val_score
            self.counter = 0
            torch.save(model.state_dict(), self.path)
            print('Val loss improved. Saved model.')
            return False
        else:
            self.counter += 1
            print(f'Early stopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                print(f'Stopped early. Best val loss: {self.best_score:.4f}')
                return True

In [21]:
def train_one_epoch(model, train_loader, optimizer, device, criterion):
    """Train model for one epoch and return the mean train_loss."""
    model.train()
    running_loss_train = 0
    for inputs, labels in train_loader:
        labels = labels.type(torch.LongTensor)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
    train_loss = running_loss_train / len(train_loader.dataset)
    return train_loss

In [22]:
def validate(model, valid_loader, device, criterion):
    """Validate model and return the accuracy and mean loss."""
    model.eval()
    correct = 0
    running_loss_val = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            labels = labels.type(torch.LongTensor)
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            pred = outputs.argmax(dim=1)
            correct += pred.eq(labels).sum().item()
            running_loss_val += loss.item()
    val_acc = correct / len(valid_loader.dataset)
    val_loss = running_loss_val / len(valid_loader.dataset)
    return val_acc, val_loss

In [23]:
def fit(model, train_loader, valid_loader, learning_rate, num_epochs):
    criterion = nn.CrossEntropyLoss(reduction='sum')
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    es = EarlyStopping(mode='min', path='./EarlyStop.pth', patience=5)
    model = model.to(device)
    scheduler = ExponentialLR(optimizer, gamma=0.1)

    for epoch in range(1, num_epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer, device, criterion)
        val_acc, val_loss = validate(model, valid_loader, device, criterion)
        scheduler.step()
        print(f'Epoch {epoch:2}/{num_epochs}',
              f'train loss: {train_loss:.4f}',
              f'val loss: {val_loss:.4f}',
              f'val acc: {val_acc:.2%}',
              sep=' | '
             )
        if es(val_loss, model):
            break

In [25]:
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 1
NUM_EPOCHS = 5
LEARNING_RATE = 1e-3
NUM_WORKERS = 0
PIN_MEMORY = False

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)
valid_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY)


model = ConvNet().to(device)
start = time.time()
fit(model, train_loader, valid_loader, learning_rate=LEARNING_RATE, num_epochs=NUM_EPOCHS)
print(f'Total training time: {time.time() - start}')
model.load_state_dict(torch.load('EarlyStop.pth'))

Epoch  1/5 | train loss: 26910.2105 | val loss: 349.1643 | val acc: 81.67%
Val loss improved. Saved model.
Epoch  2/5 | train loss: 71.4176 | val loss: 287.5189 | val acc: 85.61%
Val loss improved. Saved model.
Epoch  3/5 | train loss: 42.9539 | val loss: 296.1462 | val acc: 86.77%
Early stopping counter: 1/5
Epoch  4/5 | train loss: 40.0021 | val loss: 297.2329 | val acc: 86.72%
Early stopping counter: 2/5
Epoch  5/5 | train loss: 39.6532 | val loss: 297.3329 | val acc: 86.71%
Early stopping counter: 3/5
Total training time: 48.04467511177063


<All keys matched successfully>